[intro]

In [1]:
import subprocess
import pandas as pd
import itables
import os
import json
from ipywidgets import Dropdown
from ipywidgets import IntSlider
from ipywidgets import interact, interactive, fixed, interact_manual
import glob
from IPython.core.display import Image, display


#get top directory path of the current git repository, under the presumption that 
#the notebook was launched from within the repo directory
gitRepoPath=subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode('ascii').strip()

#move to the top of the directory
os.chdir(gitRepoPath)

#load the WMADB JSON
with open(os.path.join('dbStore','WMAnatDB.json')) as json_data:
    WMAnatDB = json.load(json_data)
    
#build components of pandas spreadsheet
import pandas as pd
#stored as dict key pairing
#dois=[WMAnatDB[x]['doi']['1'] for x in WMAnatDB.keys()]
#stored as dict key pairing
curators=[WMAnatDB[x]['curator']['1'] for x in WMAnatDB.keys()]
#stored as list
titles=[WMAnatDB[x]['title'][0] for x in WMAnatDB.keys()]
tractNums=[len(WMAnatDB[x]['tractDepictions']) for x in WMAnatDB.keys()]
years=[WMAnatDB[x]['published']['date-parts'][0][0] for x in WMAnatDB.keys()]
authors=[WMAnatDB[x]['author'][0]['given'] + ' ' + WMAnatDB[x]['author'][0]['family'] for x in WMAnatDB.keys()]

#make the spreadsheet
dataStructure={'title': titles, 'firstAuthor': authors, 'year': years, 'numOfTracts': tractNums, 'curator': curators}
overviewFrame=pd.DataFrame(data=dataStructure)

#use itables to make it nice and interactive
import itables
itables.show(overviewFrame)

<IPython.core.display.Javascript object>

In [4]:
#create title stems or dropdown menu
titleStems=[WMAnatDB[x]['title'][0][0:50] for x in WMAnatDB.keys()]
#create a index pairing
dropDownDocList=list(zip(titleStems, list(range(len(titleStems)))))
listIndex=Dropdown(options=dropDownDocList, value=1, description="Article Title Stem")

def changeDoc(*args):
    tractIndex.options=list(zip( \
                    [WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][x]['tractTermsUsed']['1'] for x in list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())], \
                    list(range(len([WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][x]['tractTermsUsed']['1'] for x in list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())]))) \
                    ))
listIndex.observe(changeDoc, 'value')

#absolutely _brutal_ way to make option list from the DB
tractIndex=Dropdown(options= list(zip( \
                    [WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][x]['tractTermsUsed']['1'] for x in list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())], \
                    list(range(len([WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][x]['tractTermsUsed']['1'] for x in list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())]))) \
                    )), value=2, description="tractNames")

def changeTract(*args):
    figureSlider.max=len(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())[tractIndex.value]]['figures'])
    descripSlider.max=len(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())[tractIndex.value]]['descriptions'])                   

tractIndex.observe(changeTract, 'value')
    
figureSlider=IntSlider( value=1, min=1, max= \
                       len(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())[tractIndex.value]]['figures']), \
                       step=1, description='Figure Index', continuous_update=False)

descripSlider=IntSlider( value=1, min=1, max= \
                       len(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'][list(WMAnatDB[list(WMAnatDB.keys())[listIndex.value]]['tractDepictions'].keys())[tractIndex.value]]['descriptions']), \
                       step=1, description='Description Index', continuous_update=False)

def displayInteractFigure(articleIndex, tractIndex, figIndex):
    from pyWMAD import DB_JSONparsing
    figOut=DB_JSONparsing.extractTractFigureDB
from pyWMAD import DB_JSONparsing
interact(DB_JSONparsing.extractTractFigureDB, articleIndex=listIndex, tractIndex=tractIndex, figIndex=figureSlider)



interactive(children=(Dropdown(description='Article Title Stem', index=1, options=(('Estimating Fiber Orientat…

<function pyWMAD.DB_JSONparsing.extractTractFigureDB(articleIndex, tractIndex, figIndex)>

In [5]:
interact(DB_JSONparsing.extractTractDescriptionDB, articleIndex=listIndex, tractIndex=tractIndex, descripIndex=descripSlider)


interactive(children=(Dropdown(description='Article Title Stem', index=1, options=(('Estimating Fiber Orientat…

<function pyWMAD.DB_JSONparsing.extractTractDescriptionDB(articleIndex, tractIndex, descripIndex)>

In [4]:
pwd

'D:\\Documents\\gitDir\\WMAD'